## Testing giving all scores of same value same rank

In [11]:
# (protein, score, position)
correct_position = 3
scores = [
    ('insulin', 2, 3),
    ('bdnf', 2, 3),
    ('caprin', 2, 3),
    ('insulin', 0.5, 1),
    ('caprin', 0, 4)
]

In [12]:
def give_ranks(scores):
    ranking = {}
    scores.sort(reverse=True, key=lambda x: x[1])
    # if scores are all the same, don't increment the rank
    rank = 0
    last_score = None
    for score in scores:
        if int(score[1]) != last_score:
            rank += 1
            last_score = int(score[1])
        if int(score[2]) == correct_position:
            ranking[score[0]] = rank
    return ranking

In [13]:
give_ranks(scores)

{'insulin': 1, 'bdnf': 1, 'caprin': 1}